<a href="https://colab.research.google.com/github/ZorkDaNerd/CS345-Text-Recognition/blob/main/Text_Recognition_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*This notebook is part of our text recognition project for class CS345 at Colorado State University.
Original versions were created by Zachary Shimpa, Jenelle Dobyns and Jordan Rust.
The content is availabe [on GitHub](github.com/ZorkDaNerd/CS345-Text-Recognition).*

*Code help and referance was provided from Prof. Asa Ben-Hur and CS 345: Machine Learning Foundations and Practice at Colorado State University.
Original versions of these notebooks were created by Asa Ben-Hur with updates by Ross Beveridge.
The content is availabe [on his GitHub](https://github.com/asabenhur/CS345).*

<a href="https://colab.research.google.com/github/ZorkDaNerd/CS345-Text-Recognition/blob/main/Text_Recognition_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Possible data sets

https://github.com/amephraim/nlp/tree/master/code 
Dataset from this

https://medium.com/mlearning-ai/sentiment-analysis-using-lstm-21767a130857

https://en.wikipedia.org/wiki/Sentiment_analysis



# Description of Project

This project is about recognizing text emotions using LSTM. This is a form of natural language processing.

### Coding languages and packages used in project

Ex: Anaconda, Python, ect

In [ ]:
#Import
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
from glob import glob
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import string
import re

In [ ]:
# Let's import our data. No train/test split yet - we have some data pre-processing to do!
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    as_supervised=True)

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
# It's time to eliminate words and characters that don't contribute to the overall sentiment of our reviews
# Let's import stopwords and parse our reviews to remove these unnecessary characters and words (like 'and', 'a', '!', etc.)

import nltk
from nltk.corpus import stopwords
import string
def text_processing(text):
    nopunc = []
    for char in text:
        if char not in string.punctuation:
            if char!=str("0") and char!=str("1"):
                nopunc.append(char)
    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

# Project Code

In [ ]:
#Defaults
book = []
thinBook = []
uselessWords = ["the", "and", "a", "to", "of", "i", "in", "a", "there"]

This section puts all of the books into 2 numpy arrays, 1 fore the origional books and 1 for the thined version.

In [ ]:
for file in glob('texts\*.txt'):
    book.append(open(file, 'r').read())
    thinBook.append(open(file, 'r').read())
book = np.array(book)
thinBook = np.array(thinBook)

This section removes all of the unwanted words from the books and makes the words lowercase.

In [ ]:
for numb in range(len(thinBook)):
    thinBook[numb] = thinBook[numb].translate(str.maketrans('', '', string.punctuation)).lower()
    editBook = thinBook[numb].split()
    thinedBook = [word for word in editBook if word not in uselessWords]
    thinBook[numb] = ' '.join(thinedBook) 

How many words and sentances are in the books? and how long would they take to read?

In [ ]:
for numBook in range(len(book)):
    wordCont = len(book[numBook].strip().split(" "))
    sentCont = len(re.split(r'[.!?]+', book[numBook]))
    timeRead = np.ceil((wordCont / 200)/60)
    print("Book " + str(numBook + 1) + " has " + str(wordCont) + " words, " + str(sentCont) + " sentences, and will take " + str(timeRead) + " hours to read.")  

Book 1 has 70804 words, 6760 sentences, and will take 6.0 hours to read.
Book 2 has 78053 words, 7410 sentences, and will take 7.0 hours to read.
Book 3 has 95546 words, 9409 sentences, and will take 8.0 hours to read.
Book 4 has 196707 words, 18953 sentences, and will take 17.0 hours to read.


# Findings

# Analysis of results